<a href="https://colab.research.google.com/github/LeonardoViotti/cdr-training/blob/develop/notebooks/aggregated-cdr-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CDR Analysis practical exercises

This notebook contains exercises to analyze aggregated CDR data. 

It uses mock CDR data for Ghana from February to May 2020. 

# Environment set-up
Before we start, please run the cell below to set-up the environment. You can hide this section afterwards by clicking the arrow next to the title.

In [ ]:
#------------------------------------------------------------------------
# Libraries installation

!pip install geopandas

#------------------------------------------------------------------------
# User defined functions

def time_complete(data, timefreq = 'D'):
    data = data.reset_index()
    timevar = 'date'
    data[timevar] = data[timevar].astype('datetime64[D]')
    full_time_range = pd.date_range(data[timevar].min(),  
                                            data[timevar].max(), 
                                            freq = timefreq)
    data = data.set_index(timevar)
    data = data.reindex(full_time_range,  fill_value=0)
    data.index.name = 'date'
    return(data)

def day_lag(df):
    # Makse sure date is datetime type
    df['date'] = pd.to_datetime(df['date'])
    
    # Sort by region and date
    df = df.sort_values(['date'])
    
    # Lag value
    df['value_l'] = df['value'].shift(1)
    
    # Drop values if missing dates
    df['value_l'] = df['value_l'].where(df.date.diff() == dt.timedelta(days = 1), np.nan)
    
    return df


# Let's start
First let's import the packages we will use:

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

import datetime as dt

Now, import the datasets we will use on this exercise using the cell below.

In [ ]:
from google.colab import files
files.upload()

We can check if all files were loaded into Colab by running the cell below. You should see the following files:
 - admin1.geojson
 - movements_per_day.csv
 - subscribers_per_day.csv
 - transactions_per_day.csv

In [ ]:
!ls

Before we start, let's load and have a look at each one of the indicators.

In [ ]:
# Load transactions per day data
trans = pd.read_csv('transactions_per_day.csv')
trans.head()

In [ ]:
# Load subscribers per day data
subs = pd.read_csv('subscribers_per_day.csv')
subs.head()

In [ ]:
# Load movements
mov = pd.read_csv('movements_per_day.csv')
mov.head()

## Exercise 1 - Quality checks

First step is to take a quick look at the completeness and consistency of the data.

- Check the number of regions per day
- Check time completeness of the series
- Compare number of subscribers to the number of calls


### Part 1 instructions:


1. Aggregate the transactions data by `date` using the `groupby()` and `agg()` methods. Store the result in a new DataFrame called `trans_day`
    - Calculate the number of unique regions per day (TIP: use the `pd.Series.nunique` function.)
    - Calculate sum of total transactions per day in the country.

2. Slice the `trans_day` DataFrame keeping only rows where there are fewer than 16 regions. Is there any?

3. Run the time complete function (user defined) to create a new DataFrame replacing missing rows with zeros:
    `trans_day_tcomplete = time_complete(trans_day)`

4. Plot the `value` column of the time complete data frame.
 - TIP: you can use the `plot()` method of the DataFrame, but pass `y = 'value'` a argument in order not to plot the `region` column also. 

Do you see anything unusual?

### Solutions Part 1:

In [ ]:
# 1. Aggregate data by day summing values across all regions
trans_day = trans.groupby('date').agg({'region': pd.Series.nunique,
                                       'value': np.sum})

In [ ]:
#2.  Test if any date has fewer than 16 regions
trans_day[trans_day['region'] < 16]

In [ ]:
# PLot
trans_day_tcomplete = time_complete(trans_day)
trans_day_tcomplete.plot(y = 'value')

###  Part 2 instructions:



1. Merge `subs` and `trans` using `['region', 'date']` columns as keys.

    TIP: use the `suffixes` argument to differentiate the values on from each DataFrame 


2. Use the `plot.scatter()` method on the merged DataFrame to compare the values of the two columns.
 - Set the x axis as the number of subscribers
 - Set the y axis as the number of transactions

### Solutions part 2:

In [ ]:
# Merge the two DataFrames
mdf = subs.merge(trans, 
           on = ['region', 'date'], 
           suffixes = ('_trans', '_subs'))

mdf.plot.scatter(x = 'value_subs',
                 y = 'value_trans')

## Exercise 2 - Changes over time

Now let's look how movement has changed over time. 

For simplicity we will use country level data and only look at movements between two different regions. Here's a quick summary of the comparisons we'll do:

- Absolute values
- Change from previous day
- Change from Baseline (defined as the average from February 1st to March 15th)

Also here is a time-line to help interpret the results:
- February 1st to March 15th: Baseline period
- March 16th: initial restrictions imposed
- March 30th: Lockdown measures on parts of Accra and Kumasi metropolitan areas

### Instructions

1. On the `mov` DataFrame, remove rows where users move within a region.
 - TIP: slice the the DataFrame keeping only rows that match the condition `mov['region_from'] != mov['region_to']`

2. Aggregate `mov` DataFrame by `date` to have country level data using the `groupby()` and `agg()` methods.

3. Use the `day_lag()` function to create a DataFrame with a column containing the value of movements from the previous day.

`mov_day = day_lag(mov_day)`

4. Use the `bl_values()` function to create a DataFrame with a column containing the average number of movements in the baseline.

`mov_day = bl_values(mov_day)`

Before we move to part 4, lets review what the `bl_value()` function is doing.

In [ ]:
def bl_values(df):
    # Makse sure date is datetime type
    df['date'] = pd.to_datetime(df['date'])

    # Create weekday variable to calculate baseline values
    df['weekday'] = df['date'].dt.dayofweek

    # Keep only entries from Feb 1st to Mar 15th
    bl = df[df['date'] < dt.datetime(2020, 3, 16)]

    # Calculate baseline averages for each weekday
    bl_averages = bl.groupby(['weekday']).agg({'value': np.mean}).reset_index()
    
    # Merge bl averages as a column on original df
    ndf = df.merge(bl_averages, on = ['weekday'],
                  suffixes = ('', '_bl')).drop('weekday', axis = 1)

    return ndf


5. Create two columns containing:
 - Percent changes from previous day; and
 - Percent change from baseline columns.


6. Create 3 different line plots:
    - Level values of total movements
    - Percent change from previous day
    - Percent change from baseline

TIP: You can again use the `plot()` method keeping sure to specify the x and y axis.

How does these 3 plots compare?

### Solutions

In [ ]:
# 1. Remove movements in the same district
mov = mov[mov['region_from'] != mov['region_to']]

# 2. Aggregate by day
mov_day = mov.groupby('date').agg({'value' : np.sum}).reset_index()

# 3
mov_day = day_lag(mov_day)

# 4
mov_day = bl_values(mov_day)

# # 5. Calculate percent change columns
mov_day['p_change_l'] = (mov_day['value'] - mov_day['value_l'])/mov_day['value_l']

mov_day['p_change_bl'] = (mov_day['value'] - mov_day['value_bl'])/mov_day['value_bl']

mov_day

In [ ]:
ax = mov_day.plot(x = 'date', y = 'value')
ax.axvline('2020-03-16', color='orange', linestyle='--', lw=1)
ax.axvline('2020-03-30', color='red', linestyle='--', lw=1)

In [ ]:
ax = mov_day.plot(x = 'date', y = 'p_change_l')
ax.axvline('2020-03-16', color='orange', linestyle='--', lw=1)
ax.axvline('2020-03-30', color='red', linestyle='--', lw=1)


In [ ]:
ax = mov_day.plot(x = 'date', y = 'p_change_bl')
ax.axvline('2020-03-16', color='orange', linestyle='--', lw=1)
ax.axvline('2020-03-30', color='red', linestyle='--', lw=1)



## Exercise 3 - Choropleth
As a final exercise we will create a map using the `admin1.geojson` to display changes in the changes on the average number of daily trips between districts since baseline.

To start run the cell bellow for the initial set-up:

In [ ]:
# Load choropleth package
import folium

# Load the geoson in two ways
gdf = gpd.read_file("admin1.geojson")


Let's see how the geometries data looks: 

In [ ]:
gdf.head()

In [ ]:
# Plot the administrative boundaries
gdf.plot()


Let's create a region level DataFrame containing pre and post baseline movement levels:

In [ ]:
def create_region_df(df, region = 'region_from'):
    
    # Makse sure date is datetime type
    df['date'] = pd.to_datetime(df['date'])

    # Keep only entries from Feb 1st to Mar 15th
    bl = df[df['date'] < dt.datetime(2020, 3, 16)]
    
    # Post baseline
    post = df[df['date'] >= dt.datetime(2020, 3, 16)]

    # Calculate baseline 
    bl_movs = bl.groupby([region]).agg({'value': np.mean}).reset_index()
    
    # Calculate region df
    rdf = post.groupby([region]).agg({'value': np.mean}).reset_index()

    
    # Merge bl averages as a column on original df
    ndf = rdf.merge(bl_movs, on = [region],
                  suffixes = ('_pos', '_bl'))
    
    # Percent change
    ndf['p_change'] = (ndf['value_pos'] - ndf['value_bl'])/ndf['value_bl']
    
    
    
    return ndf.rename(columns = {region : 'region'})

mov_region = create_region_df(mov)

Now, combine the GeoDataFrame and the `mov_region` DataFrame so our interest data and the geometries are in the same table.

In [ ]:
gdf = gdf.merge(mov_region, on = 'region')
gdf.head()

Let's load a base map in Ghana.

In [ ]:

# Load base map
gmap = folium.Map(location=[7.28, -0.97], zoom_start=7)
gmap

Finally, lets use `gdf` to add a layer to the map: 

In [ ]:

# Create choropleth on top of map
choropleth = folium.Choropleth(
    gdf,
    data=gdf,
    key_on="properties.region",
    columns=["region", "p_change"],
    fill_color="RdPu",
    fill_opacity=0.7,
    line_opacity=0.2).add_to(gmap)

# Add hover information
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name', 'p_change'])
)

# Print
gmap